# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ingeniero jacobacci,-41.3427,-69.5448,5.45,72,100,1.55,AR,1717516562
1,1,songyang,34.4553,113.0281,20.09,63,100,1.35,CN,1717516563
2,2,bilibino,68.0546,166.4372,7.49,68,85,0.93,RU,1717516563
3,3,cooper's town,26.8667,-77.5167,25.87,81,25,5.67,BS,1717516563
4,4,sidney,-33.8679,151.2073,12.62,74,93,3.09,AU,1717516395


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_df = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot_df

C:\Users\hobby\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
small_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
small_city_df = small_city_df.dropna()

small_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
135,135,guaranesia,-21.2992,-46.8025,26.67,27,0,2.40,BR,1717516588
189,189,pacific grove,36.6177,-121.9166,22.72,80,0,4.02,US,1717516598
257,257,spokoynaya,44.2468,41.4015,23.32,74,0,2.49,RU,1717516610
260,260,ambovombe,-25.1667,46.0833,22.02,69,0,3.12,MG,1717516611
300,300,saint-francois,46.4154,3.9054,23.71,54,0,1.03,FR,1717516618
384,384,colonia,50.9333,6.9500,24.42,47,0,2.57,DE,1717516547
387,387,kununurra,-15.7667,128.7333,21.99,56,0,4.12,AU,1717516634
455,455,chulakkurgan,43.7650,69.1814,22.47,17,0,3.85,KZ,1717516649
463,463,chiredzi,-21.0500,31.6667,24.99,33,0,0.00,ZW,1717516650
487,487,houlton,46.1262,-67.8403,23.73,56,0,4.12,US,1717516654


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = small_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
135,guaranesia,BR,-21.2992,-46.8025,27,
189,pacific grove,US,36.6177,-121.9166,80,
257,spokoynaya,RU,44.2468,41.4015,74,
260,ambovombe,MG,-25.1667,46.0833,69,
300,saint-francois,FR,46.4154,3.9054,54,
384,colonia,DE,50.9333,6.9500,47,
387,kununurra,AU,-15.7667,128.7333,56,
455,chulakkurgan,KZ,43.7650,69.1814,17,
463,chiredzi,ZW,-21.0500,31.6667,33,
487,houlton,US,46.1262,-67.8403,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guaranesia - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
spokoynaya - nearest hotel: No hotel found
ambovombe - nearest hotel: La source
saint-francois - nearest hotel: Chez Lily
colonia - nearest hotel: Wasserturm Hotel Cologne
kununurra - nearest hotel: No hotel found
chulakkurgan - nearest hotel: адеми
chiredzi - nearest hotel: Nesbitt Arms Hotel
houlton - nearest hotel: Shiretown Inn & Suites
beatrice - nearest hotel: Victorian Inn Beatrice


,City,Country,Lat,Lng,Humidity,Hotel Name
135,guaranesia,BR,-21.2992,-46.8025,27,No hotel found
189,pacific grove,US,36.6177,-121.9166,80,Pacific Grove Inn
257,spokoynaya,RU,44.2468,41.4015,74,No hotel found
260,ambovombe,MG,-25.1667,46.0833,69,La source
300,saint-francois,FR,46.4154,3.9054,54,Chez Lily
384,colonia,DE,50.9333,6.9500,47,Wasserturm Hotel Cologne
387,kununurra,AU,-15.7667,128.7333,56,No hotel found
455,chulakkurgan,KZ,43.7650,69.1814,17,адеми
463,chiredzi,ZW,-21.0500,31.6667,33,Nesbitt Arms Hotel
487,houlton,US,46.1262,-67.8403,56,Shiretown Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# %%capture --no-display

map_plot_df = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    color="City",
    tiles="OSM",
    hover_cols=["Hotel Name"],
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot_df

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)